In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import (
    VGG16, VGG19, ResNet50, ResNet50V2, ResNet101, ResNet101V2, ResNet152, ResNet152V2,
    InceptionV3, InceptionResNetV2, Xception,
    MobileNet, MobileNetV2,
    DenseNet121, DenseNet169, DenseNet201,
    NASNetMobile
)
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg19_preprocess
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.resnet_v2 import preprocess_input as resnetv2_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input as inceptionresnet_preprocess
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess
from tensorflow.keras.applications.mobilenet import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenetv2_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess
from tensorflow.keras.applications.nasnet import preprocess_input as nasnet_preprocess

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Set seeds for reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Constants
train_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
test_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'
BATCH_SIZE = 32
EPOCHS = 64
IMG_SIZE = (224, 224)

# Data Generators (NO AUGMENTATION)
def get_generators(preprocess_func):
    train_gen = ImageDataGenerator(
        preprocessing_function=preprocess_func
    ).flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        seed=SEED
    )

    test_gen = ImageDataGenerator(
        preprocessing_function=preprocess_func
    ).flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )
    return train_gen, test_gen

# Build classifier head
def build_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

# Models Dictionary
models = {
    'Xception': (Xception(weights='imagenet', include_top=False, input_shape=(224,224,3)), xception_preprocess),
    'VGG16': (VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3)), vgg_preprocess),
    'VGG19': (VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3)), vgg19_preprocess),
    'ResNet50': (ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnet_preprocess),
    'ResNet50V2': (ResNet50V2(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnetv2_preprocess),
    'ResNet101': (ResNet101(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnet_preprocess),
    'ResNet101V2': (ResNet101V2(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnetv2_preprocess),
    'ResNet152': (ResNet152(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnet_preprocess),
    'ResNet152V2': (ResNet152V2(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnetv2_preprocess),
    'InceptionV3': (InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3)), inception_preprocess),
    'InceptionResNetV2': (InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3)), inceptionresnet_preprocess),
    'MobileNet': (MobileNet(weights='imagenet', include_top=False, input_shape=(224,224,3)), mobilenet_preprocess),
    'MobileNetV2': (MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3)), mobilenetv2_preprocess),
    'DenseNet121': (DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3)), densenet_preprocess),
    'DenseNet169': (DenseNet169(weights='imagenet', include_top=False, input_shape=(224,224,3)), densenet_preprocess),
    'DenseNet201': (DenseNet201(weights='imagenet', include_top=False, input_shape=(224,224,3)), densenet_preprocess),
    'NASNetMobile': (NASNetMobile(weights='imagenet', include_top=False, input_shape=(224,224,3)), nasnet_preprocess),
}

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2)

# Result storage
report_rows = []

# Training and Evaluation Loop
for model_name, (base_model, preprocess_func) in models.items():
    print(f"\n🔧 Training model: {model_name}")
    train_gen, test_gen = get_generators(preprocess_func)
    model = build_model(base_model)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(
        train_gen,
        epochs=EPOCHS,
        validation_data=test_gen,
        callbacks=[early_stop, reduce_lr],
        verbose=2
    )

    predictions = model.predict(test_gen, verbose=0)
    predicted_labels = (predictions > 0.5).astype(int).ravel()
    true_labels = test_gen.classes

    # Accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)

    # Classification Report
    report = classification_report(true_labels, predicted_labels, digits=6, output_dict=True)

    row = {
        'Model': model_name,
        'Accuracy': accuracy,
        'Macro Precision': report['macro avg']['precision'],
        'Macro Recall': report['macro avg']['recall'],
        'Macro F1': report['macro avg']['f1-score'],
        'Weighted Precision': report['weighted avg']['precision'],
        'Weighted Recall': report['weighted avg']['recall'],
        'Weighted F1': report['weighted avg']['f1-score'],
    }

    report_rows.append(row)

# Save to CSV
df_report = pd.DataFrame(report_rows)
df_report.to_csv('/kaggle/working/classification_summary.csv', index=False)
print("✅ Saved summary to /kaggle/working/classification_summary.csv")
print("📊 Classification Summary:")
print(df_report)


2025-07-20 10:11:36.815909: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753006297.012685      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753006297.069608      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1753006309.998952      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
171317808/171317808 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
234698864/234698864 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
234545216/234545216 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
51877672/51877672 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
19993432/19993432 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

🔧 Training model: Xception
Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64


I0000 00:00:1753006369.784366      57 service.cc:148] XLA service 0x7c7068003ca0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753006369.785399      57 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753006370.727537      57 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753006375.363398      57 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

Epoch 1/64
163/163 - 125s - 769ms/step - accuracy: 0.9396 - loss: 0.1548 - val_accuracy: 0.8189 - val_loss: 0.4318 - learning_rate: 0.0010
Epoch 2/64
163/163 - 54s - 331ms/step - accuracy: 0.9737 - loss: 0.0726 - val_accuracy: 0.8510 - val_loss: 0.4265 - learning_rate: 0.0010
Epoch 3/64
163/163 - 52s - 322ms/step - accuracy: 0.9806 - loss: 0.0562 - val_accuracy: 0.8478 - val_loss: 0.4458 - learning_rate: 0.0010
Epoch 4/64
163/163 - 53s - 322ms/step - accuracy: 0.9806 - loss: 0.0506 - val_accuracy: 0.7644 - val_loss: 0.9168 - learning_rate: 0.0010
Epoch 5/64
163/163 - 54s - 332ms/step - accuracy: 0.9875 - loss: 0.0355 - val_accuracy: 0.8013 - val_loss: 0.7116 - learning_rate: 2.0000e-04

🔧 Training model: DenseNet201
Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64
163/163 - 142s - 869ms/step - accuracy: 0.9521 - loss: 0.1238 - val_accuracy: 0.8574 - val_loss: 0.4089 - learning_rate: 0.0010
Epoch 2/64
163/163 - 53s - 325ms/step - accuracy: 0.9682 - loss: 0.0766 - val_accuracy: 0.7901 - val_loss: 0.7076 - learning_rate: 0.0010
Epoch 3/64
163/163 - 53s - 326ms/step - accuracy: 0.9797 - loss: 0.0573 - val_accuracy: 0.8173 - val_loss: 0.5878 - learning_rate: 0.0010
Epoch 4/64
163/163 - 53s - 326ms/step - accuracy: 0.9877 - loss: 0.0363 - val_accuracy: 0.8381 - val_loss: 0.5312 - learning_rate: 2.0000e-04

🔧 Training model: NASNetMobile
Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64
163/163 - 116s - 713ms/step - accuracy: 0.9294 - loss: 0.1700 - val_accuracy: 0.8077 - val_loss: 0.4794 - learning_rate: 0.0010
Epoch 2/64
163/163 - 51s - 311ms/step - accuracy: 0.9565 - loss: 0.1068 - val_accuracy: 0.7788 - val_loss: 0.6543 - learning_rate: 0.0010
Epoch 3/64
163/163 - 51s - 312ms/step - accuracy: 0.9657 - loss: 0.0896 - val_accuracy: 0.8429 - val_loss: 0.4233 - learning_rate: 0.0010
Epoch 4/64
163/163 - 50s - 309ms/step - accuracy: 0.9588 - loss: 0.0969 - val_accuracy: 0.8446 - val_loss: 0.4435 - learning_rate: 0.0010
Epoch 5/64
163/163 - 51s - 313ms/step - accuracy: 0.9682 - loss: 0.0812 - val_accuracy: 0.8654 - val_loss: 0.3951 - learning_rate: 0.0010
Epoch 6/64
163/163 - 52s - 318ms/step - accuracy: 0.9749 - loss: 0.0676 - val_accuracy: 0.8109 - val_loss: 0.6357 - learning_rate: 0.0010
Epoch 7/64
163/163 - 51s - 310ms/step - accuracy: 0.9747 - loss: 0.0661 - val_accuracy: 0.8237 - val_loss: 0.6377 - learning_rate: 0.0010
Epoch 8/64
163/163 - 51s - 312ms/